# Création du Dataset en scrappant les infos depuis une liste d'url

## Imports

In [1]:
import scraping_info_perfume as sip
import pandas as pd
import numpy as np
import time

df = pd.read_csv('list_url_by_popular_designer.csv')
Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
urls_deja_scrapees = Fragrantica_dataset['url'].tolist()

# Choix de la lettre à scraper
lettre = ('G')
df_a_scraper = df[df['Marque'].str.startswith(lettre)]
# Exclure les lignes déjà présentes dans Fragrantica_dataset
df_a_scraper = df_a_scraper[~df_a_scraper['Lien du Parfum'].isin(urls_deja_scrapees)] 
df_a_scraper.describe()

,Marque,Parfum,Lien du Parfum
count,1075,1075,1075
unique,5,1061,1075
top,Guerlain,Insolence Eau de Parfum,https://www.fragrantica.com/perfume/Giorgio-Ar...
freq,542,3,1


## Test pour voir le temps d'attente optimal à scraper 50 parfums

In [ ]:
batch_size = 49

# Charger le dataset existant
Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
url_deja_scrape = set(Fragrantica_dataset['url'].tolist())

# Filtrer les URLs déjà scrapées
df_a_scraper = df_a_scraper[~df_a_scraper['Lien du Parfum'].isin(url_deja_scrape)]

# Initialiser les DataFrames pour les données scrapées
scraped_data_1 = pd.DataFrame()
scraped_data_2 = pd.DataFrame()

# Fonction pour scraper un batch et mesurer le temps d'exécution
def scrape_batch(batch, batch_num):
    liste_url = batch["Lien du Parfum"].tolist()
    start_time = time.time()
    scraped_data = sip.scraping_multi_perfume_info(liste_url)
    end_time = time.time()
    execution_time = end_time - start_time
    scraped_data.to_csv(f"scraped_data_{batch_num}.csv", index=False)
    return scraped_data, execution_time

# Scraper le premier batch
batch1 = df_a_scraper.iloc[0 : batch_size]
scraped_data_1, execution_time_1 = scrape_batch(batch1, 1)
print(f"Temps d'exécution pour batch 1 : {execution_time_1:.1f} secondes.")
print(f"Soit {execution_time_1 / batch_size:.1f} secondes par URL.")

# Attendre avant de scraper le deuxième batch
time.sleep(1200)

# Scraper le deuxième batch
batch2 = df_a_scraper.iloc[batch_size : 2 * batch_size]
scraped_data_2, execution_time_2 = scrape_batch(batch2, 2)
print(f"Temps d'exécution pour batch 2 : {execution_time_2:.1f} secondes.")
print(f"Soit {execution_time_2 / batch_size:.1f} secondes par URL.")




Exception ignored in: <function Service.__del__ at 0x000002523881C040>
Traceback (most recent call last):
  File "C:\Users\tapri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\selenium\webdriver\common\service.py", line 200, in __del__
    self.stop()
  File "C:\Users\tapri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\selenium\webdriver\common\service.py", line 157, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\tapri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\selenium\webdriver\common\service.py", line 137, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\urllib\request.py", line 214, in urlopen
    r

Temps d'exécution pour batch 1 : 566.3 secondes.
Soit 11.6 secondes par URL.


In [2]:

Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
print(f"Nombre de lignes dans Fragrantica_dataset : {Fragrantica_dataset.shape[0]}")
scraped_data1 = pd.read_csv(f"scraped_data_1.csv")
#afficher le nomre de ligne de chaque dataframe
# Concaténer les deux DataFrames
combined_data = pd.concat([Fragrantica_dataset, scraped_data1])
combined_data.drop_duplicates(subset=['url'], inplace=True)
combined_data.to_csv("Fragrantica_dataset.csv", index=False)

Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
print(f"Nombre de lignes dans Fragrantica_dataset : {Fragrantica_dataset.shape[0]}")

'''
scraped_data2 = pd.read_csv(f"scraped_data_2.csv")
combined_data = pd.concat([Fragrantica_dataset, scraped_data2])
combined_data.drop_duplicates(subset=['url'], inplace=True)
combined_data.to_csv("Fragrantica_dataset.csv", index=False)


Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
print(f"Nombre de lignes dans Fragrantica_dataset : {Fragrantica_dataset.shape[0]}")
print(f"✅ Les données ont été ajoutées à Fragrantica_dataset.csv sans doublons.")
'''



Nombre de lignes dans Fragrantica_dataset : 1675
Nombre de lignes dans Fragrantica_dataset : 1724


'\nscraped_data2 = pd.read_csv(f"scraped_data_2.csv")\ncombined_data = pd.concat([Fragrantica_dataset, scraped_data2])\ncombined_data.drop_duplicates(subset=[\'url\'], inplace=True)\ncombined_data.to_csv("Fragrantica_dataset.csv", index=False)\n\n\nFragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")\nprint(f"Nombre de lignes dans Fragrantica_dataset : {Fragrantica_dataset.shape[0]}")\nprint(f"✅ Les données ont été ajoutées à Fragrantica_dataset.csv sans doublons.")\n'

## Le script qui scrape les infos par batch de 49 lignes

In [ ]:
batch_size = 49
num_batches = len(df_a_scraper) // batch_size + (1 if len(df_a_scraper) % batch_size != 0 else 0)
print(f"📦 TAILLE DES BATCHS = {batch_size}")


#1) Obtenir les URLs déjà scrapées
print(f"📂 Nous avons déja {len(urls_deja_scrapees)} URLs dans le dataset Fragrantica.")
print("--------------------------------------------------------------------------------")


num_url_scrap = 0
#2) Itérer sur les batches 
for i in range(num_batches):
    print(f"\n📡 Traitement du batch {i+1}/{num_batches}...")

    batch = df_a_scraper.iloc[i * batch_size : (i + 1) * batch_size]
    liste_url = batch["Lien du Parfum"].tolist()

    # 🔍 Supprimer les URLs déjà scrapées
    old_len = len(liste_url)
    urls_deja_scrapees = Fragrantica_dataset['url'].tolist()
    liste_url = [url for url in liste_url if url not in urls_deja_scrapees]
    new_len = len(liste_url)
    print(f"🔍 {old_len - new_len} URLs sur {old_len} ont déjà été scrapées et retirées du batch {i+1}.")

    if not liste_url:  # Si aucun URL à scraper, passer au batch suivant
        print(f"⏭️ Batch {i+1} ignoré (toutes les URLs étaient déjà scrapées).")
        continue
    

    try:
        start_time = time.time()
        scraped_data = sip.scraping_multi_perfume_info(liste_url)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"⏱️ Temps d'exécution du scraping sur le batch{i+1} : {execution_time:.1f} secondes.")
        print(f"⏱️ Soit un total de {execution_time/len(liste_url):.1f} secondes par URL.")
        
        # Vérifier si le batch est vide

        '''
        cette condition n'est jamais remplie
        '''
        if scraped_data['nom_parfum'].empty:
            print(f"🚨 Erreur critique : Le batch {i+1} est vide. On fait une grosse pause de 1H")
            time.sleep(3601)  # Pause de 5 minutes pour éviter d'être bloqué par le site
            continue
        # Vérifier si le scrappinng c'es mal passé
        if scraped_data['nom_parfum'].isnull().sum() > 0:
            print(f"🚨 Erreur critique : Le batch {i+1} contient {scraped_data['nom_parfum'].isnull().sum()} lignes vides. On fait une grosse pause de 1H")
            # Filtrer les lignes valides
            scraped_data = scraped_data.dropna(subset=['nom_parfum'])
    
            file_name = f"scrap_batch_{i+1}.csv"
            scraped_data.to_csv(file_name, index=False) # Sauvegarde du batch sans index inutile
            print(f"✅ Batch {i+1} enregistré dans {file_name} avec les lignes valides.")
            num_url_scrap += len(scraped_data)
            time.sleep(3601)  # Pause de 5 minutes pour éviter d'être bloqué par le site
            continue
            '''        
            print(f"🚨 Erreur critique : Le batch {i+1} contient des lignes vides. Arrêt immédiat du script.")
            sys.exit(1)  # 💀 Arrêt immédiat du script avec un code d'erreur 1
            '''

        else:
            file_name = f"scrap_batch_{i+1}.csv"
            scraped_data.to_csv(file_name, index=False) # Sauvegarde du batch sans index inutile
            print(f"✅ Batch {i+1} enregistré dans {file_name}.")
            num_url_scrap += new_len

        print("On attend 30 minutes")
        time.sleep(1801)  # Pause pour éviter d'être bloqué par le site

    except Exception as e:
        print(f"⚠️ Erreur lors du scraping du batch {i+1}: {e}")

'''
Fais une grosse pause seulement si on a une 'Too many requests' erreur

'''



print(f"\n🎉 AU FINAL {num_url_scrap} URLs ont été scrapées avec succès !")

#3) Regarder le dernier batch
scraped_data = pd.DataFrame()
scraped_data.head()



📦 TAILLE DES BATCHS = 49
📂 Nous avons déja 1675 URLs dans le dataset Fragrantica.
--------------------------------------------------------------------------------

📡 Traitement du batch 1/22...
🔍 0 URLs sur 49 ont déjà été scrapées et retirées du batch 1.
⏱️ Temps d'exécution du scraping sur le batch1 : 405.2 secondes.
⏱️ Soit un total de 8.3 secondes par URL.
🚨 Erreur critique : Le batch 1 contient 48 lignes vides. On fait une grosse pause de 1H
✅ Batch 1 enregistré dans scrap_batch_1.csv avec les lignes valides.

📡 Traitement du batch 2/22...
🔍 0 URLs sur 49 ont déjà été scrapées et retirées du batch 2.
⏱️ Temps d'exécution du scraping sur le batch2 : 624.4 secondes.
⏱️ Soit un total de 12.7 secondes par URL.
✅ Batch 2 enregistré dans scrap_batch_2.csv.
On attend 30 minutes

📡 Traitement du batch 3/22...
🔍 0 URLs sur 49 ont déjà été scrapées et retirées du batch 3.
⏱️ Temps d'exécution du scraping sur le batch3 : 550.3 secondes.
⏱️ Soit un total de 11.2 secondes par URL.
✅ Batch 3 en

## Code pour ajouter nos nouvelles lignes à 'Fragrantica_dataset.csv' 

In [3]:
num_premier_batch = 1
num_dernier_batch = 19
for i in range(num_premier_batch-1, num_dernier_batch):
    scraped_data = pd.read_csv(f"scrap_batch_{i+1}.csv")
    Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")

    # Concaténer les deux DataFrames
    combined_data = pd.concat([Fragrantica_dataset, scraped_data])

    # Supprimer les doublons
    combined_data.drop_duplicates(subset=['url'], inplace=True)

    # Sauvegarder le dataset combiné
    combined_data.to_csv("Fragrantica_dataset.csv", index=False)
    print(f"✅ Les données du batch {i+1} ont été ajoutées à Fragrantica_dataset.csv sans doublons.")


✅ Les données du batch 1 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 2 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 3 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 4 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 5 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 6 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 7 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 8 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 9 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 10 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 11 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 12 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅